In [6]:
import os
import cv2
import numpy as np
import torch
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint
from models.yolo import Model
from PIL import Image, ImageEnhance
import random
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

torch.serialization.add_safe_globals([Model])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

def retinex_deillumination(image):
    """
    Apply Retinex deillumination to the image.
    :param image: Input image (numpy array in BGR format).
    :return: Image with Retinex deillumination applied (numpy array in BGR format).
    """
    # Convert the image to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    
    # Apply Gaussian blur to the L channel
    l_blur = cv2.GaussianBlur(l, (0, 0), 3)
    
    # Subtract the blurred L channel from the original L channel
    l_deilluminated = cv2.subtract(l, l_blur)
    
    # Merge the channels back and convert to BGR
    lab_deilluminated = cv2.merge((l_deilluminated, a, b))
    return cv2.cvtColor(lab_deilluminated, cv2.COLOR_LAB2BGR)

def preprocess_image(image, apply_augmentation= True, apply_clahe_flag=True, apply_retinex_flag=True):
    """
    Preprocess the image with augmentation, CLAHE, Retinex deillumination, and resizing.
    :param image: Input image (numpy array in BGR format).
    :param apply_augmentation: Whether to apply augmentation.
    :param apply_clahe_flag: Whether to apply CLAHE.
    :param apply_retinex_flag: Whether to apply Retinex deillumination.
    :return: Preprocessed image (numpy array in BGR format).
    """
    if apply_augmentation:
        image = augment_image(image)

    if apply_clahe_flag:
        image = apply_clahe(image)

    if apply_retinex_flag:
        image = retinex_deillumination(image)

    # Resize image to 640x480
    image = cv2.resize(image, (640, 480))

    return image

def load_ground_truth(label_path):
    """
    Load ground truth label from a text file.
    :param label_path: Path to the label file.
    :return: Integer ground truth value (0 for 'adl', 1 for 'fall').
    """
    with open(label_path, 'r') as file:
        line = file.readline().strip()  # Baca baris pertama dan hapus whitespace

        if not line:  # Cek jika kosong
            return 0  # Default: asumsikan 'adl' jika tidak ada data

        values = line.split()
        if len(values) == 0:  # Jika tetap kosong setelah split
            return 0  # Default: tidak ada deteksi jatuh

        first_value = values[0]

        return int(first_value)  # Konversi ke integer
import math

def calculate_vertical_speed(prev_keypoints, current_keypoints, time_elapsed):
    """
    Calculate vertical speed based on the movement of shoulders.
    :param prev_keypoints: Keypoints from the previous frame.
    :param current_keypoints: Keypoints from the current frame.
    :param time_elapsed: Time elapsed between frames (in seconds).
    :return: Vertical speed (pixels per second).
    """
    if prev_keypoints is None or time_elapsed == 0:
        return 0

    # Indices for keypoints (COCO format)
    LEFT_SHOULDER = 5
    RIGHT_SHOULDER = 6

    # Get y positions of shoulders
    prev_left_shoulder_y = prev_keypoints[LEFT_SHOULDER * 3 + 1]
    prev_right_shoulder_y = prev_keypoints[RIGHT_SHOULDER * 3 + 1]
    current_left_shoulder_y = current_keypoints[LEFT_SHOULDER * 3 + 1]
    current_right_shoulder_y = current_keypoints[RIGHT_SHOULDER * 3 + 1]

    # Calculate average y positions
    prev_shoulder_y = (prev_left_shoulder_y + prev_right_shoulder_y) / 2
    current_shoulder_y = (current_left_shoulder_y + current_right_shoulder_y) / 2

    # Calculate vertical displacement
    vertical_displacement = abs(current_shoulder_y - prev_shoulder_y)

    # Calculate vertical speed
    vertical_speed = vertical_displacement / time_elapsed
    return vertical_speed

def calculate_torso_leg_angle(keypoints):
    """
    Calculate the angle between the torso and legs.
    :param keypoints: Array of keypoints (17 keypoints, each with x, y, confidence).
    :return: Angle between torso and legs (in degrees).
    """
    # Indices for keypoints (COCO format)
    LEFT_SHOULDER = 5
    RIGHT_SHOULDER = 6
    LEFT_HIP = 11
    RIGHT_HIP = 12
    LEFT_KNEE = 13
    RIGHT_KNEE = 14

    # Get keypoints
    left_shoulder = keypoints[LEFT_SHOULDER * 3: (LEFT_SHOULDER + 1) * 3]
    right_shoulder = keypoints[RIGHT_SHOULDER * 3: (RIGHT_SHOULDER + 1) * 3]
    left_hip = keypoints[LEFT_HIP * 3: (LEFT_HIP + 1) * 3]
    right_hip = keypoints[RIGHT_HIP * 3: (RIGHT_HIP + 1) * 3]
    left_knee = keypoints[LEFT_KNEE * 3: (LEFT_KNEE + 1) * 3]
    right_knee = keypoints[RIGHT_KNEE * 3: (RIGHT_KNEE + 1) * 3]

    # Calculate midpoints
    shoulder_midpoint = ((left_shoulder[0] + right_shoulder[0]) / 2, (left_shoulder[1] + right_shoulder[1]) / 2)
    hip_midpoint = ((left_hip[0] + right_hip[0]) / 2, (left_hip[1] + right_hip[1]) / 2)
    knee_midpoint = ((left_knee[0] + right_knee[0]) / 2, (left_knee[1] + right_knee[1]) / 2)

    # Calculate vectors
    torso_vector = (hip_midpoint[0] - shoulder_midpoint[0], hip_midpoint[1] - shoulder_midpoint[1])
    leg_vector = (knee_midpoint[0] - hip_midpoint[0], knee_midpoint[1] - hip_midpoint[1])

    # Calculate dot product and magnitudes
    dot_product = torso_vector[0] * leg_vector[0] + torso_vector[1] * leg_vector[1]
    torso_magnitude = math.sqrt(torso_vector[0] ** 2 + torso_vector[1] ** 2)
    leg_magnitude = math.sqrt(leg_vector[0] ** 2 + leg_vector[1] ** 2)

    # Calculate angle in radians and convert to degrees
    angle_radians = math.acos(dot_product / (torso_magnitude * leg_magnitude))
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees

def detect_fall(keypoints, prev_keypoints, time_elapsed, speed_threshold=100, angle_threshold=45):
    """
    Detect fall based on keypoints, vertical speed, and torso-leg angle.
    :param keypoints: Array of keypoints (17 keypoints, each with x, y, confidence).
    :param prev_keypoints: Keypoints from the previous frame.
    :param time_elapsed: Time elapsed between frames (in seconds).
    :param speed_threshold: Threshold for vertical speed (pixels per second).
    :param angle_threshold: Threshold for torso-leg angle (degrees).
    :return: True if fall is detected, False otherwise.
    """
    # Check vertical speed
    vertical_speed = calculate_vertical_speed(prev_keypoints, keypoints, time_elapsed)
    if vertical_speed > speed_threshold:
        return True

    # Check torso-leg angle
    torso_leg_angle = calculate_torso_leg_angle(keypoints)
    if torso_leg_angle < angle_threshold:
        return True

    return False


# Inisialisasi variabel untuk menyimpan keypoints dari frame sebelumnya
prev_keypoints = None
time_elapsed = 1 / 30  # Asumsi frame rate 30 fps

for filename in os.listdir(test_images_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(test_images_path, filename)
        label_path = os.path.join(test_labels_path, filename.replace(".jpg", ".txt").replace(".png", ".txt"))
        
        image = cv2.imread(image_path)
        image = preprocess_image(image, apply_augmentation=True, apply_clahe_flag=True, apply_retinex_flag=True)
        
        image_resized = letterbox(image, 960, stride=64, auto=True)[0]
        image_tensor = transforms.ToTensor()(image_resized)
        image_tensor = torch.tensor(np.array([image_tensor.numpy()]))
        
        if torch.cuda.is_available():
            image_tensor = image_tensor.half().to(device)
        
        with torch.no_grad():
            output, _ = model(image_tensor)
            output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
            output = output_to_keypoint(output)
        
        fall_detected = 0
        for idx in range(output.shape[0]):
            keypoints = output[idx, 7:].T
            if detect_fall(keypoints, prev_keypoints, time_elapsed):
                fall_detected = 1
                break

        # Update previous keypoints
        prev_keypoints = keypoints
        
        predictions_list.append(fall_detected)
        ground_truth_list.append(load_ground_truth(label_path))

# Calculate metrics
precision = precision_score(ground_truth_list, predictions_list)
recall = recall_score(ground_truth_list, predictions_list)
f1 = f1_score(ground_truth_list, predictions_list)
accuracy = accuracy_score(ground_truth_list, predictions_list)

# Calculate specificity
tn, fp, fn, tp = confusion_matrix(ground_truth_list, predictions_list).ravel()
specificity = tn / (tn + fp)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Accuracy: {accuracy:.2f}")
print(f"Specificity: {specificity:.2f}")

Precision: 0.73
Recall: 0.82
F1-score: 0.77
Accuracy: 0.71
Specificity: 0.54
